In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Time steps
N = 50
T_trans = 20
T_avg = 150

# Mackey Glass constants
a = 0.2
b = 0.1
n = 10
tau = 17

# Discretisation
h = tau/(N-1)

In [3]:
def mackeyglass(x, x_lag):
    dxdt = (a * x_lag) / (1 + x_lag**n) - b * x
    return dxdt

def partialF_partialx(x):
    return -b

def partialF_partialxlag(x_lag):
    return -a * n * x_lag**n * (1 + x_lag**n)**(-2) + a * (1 + x_lag**n)**(-1)

def iter_rk4(x_prev, x_lag, f):   # for single variable 
    
    z1 = x_prev
    z2 = x_prev + (h/2)*f(z1, x_lag)
    z3 = x_prev + (h/2)*f(z2, x_lag)
    z4 = x_prev + h*f(z3, x_lag)
    
    z = (h/6)*(f(z1, x_lag) + 2*f(z2, x_lag) + 2*f(z3, x_lag) + f(z4, x_lag))
    x_curr = x_prev + z
    
    return x_curr

def euler(x_prev, x_lag, f):
    return x_prev + h*f(x_prev, x_lag)

def gram_schmidt(dx):   # matrix with vectors stored as columns
    alpha = np.zeros(shape=(N,))
    for j in range(N):
        for s in range(j):
            dx[: , j] = dx[: , j] - np.dot(dx[: , s], dx[: , j]) * dx[: , s]
        alpha[j] = np.linalg.norm(dx[: , j])
        dx[:, j] = dx[: , j] / alpha[j]
    return dx, alpha

In [4]:
deltas = 10**(-6) * np.ones(shape=(N,N)) #* np.random.uniform(low=0, high=1, size=(N,N))
x0 = 1.2

np.savetxt('deltas', deltas)

Lambda = np.zeros(shape=(N,))
dx = deltas + np.identity(N)
x = x0 * np.ones(shape=(N,))

x_ls = []
for k in range(T_trans + T_avg):
    
    for j in range(N):
        # Solving linearised DDE - RK4 for each time step in the kth interval
        dx_dot = lambda dx, dx_lag: partialF_partialx(x[N-1])*dx + partialF_partialxlag(x[0])*dx_lag
        dx[0, j] = iter_rk4(dx[N-1, j], dx[0, j], dx_dot)
        for i in range(1, N):
            dx_dot = lambda dx, dx_lag: partialF_partialx(x[i-1])*dx + partialF_partialxlag(x[i])*dx_lag
            dx[i, j] = iter_rk4(dx[i-1, j], dx[i, j], dx_dot)
    
    # Solving DDE - RK4 for each time step in the kth interval
    x[0] = iter_rk4(x[N-1], x[0], mackeyglass)
    for i in range(1,N):
        x[i] = iter_rk4(x[i-1], x[i], mackeyglass)
        x_ls.append(x[i])
        
    # Orthonormalise to get dx = orthonormalised vectors, alpha = length of vectors
    dx, alpha = gram_schmidt(dx)
    
    if k > T_trans:
        Lambda = Lambda + np.log(alpha)        

Lambda = Lambda/(T_avg*tau)

In [5]:
Lambda

array([ 0.00597657,  0.00045548, -0.03876146, -0.04988842, -0.07240083,
       -0.08741687, -0.09896995, -0.10557035, -0.11881322, -0.12170615,
       -0.12961377, -0.13222822, -0.14107948, -0.1434464 , -0.14714577,
       -0.15353482, -0.15413836, -0.15858067, -0.16200174, -0.1636146 ,
       -0.16761156, -0.17024309, -0.17229583, -0.1738435 , -0.17668117,
       -0.17858795, -0.17908252, -0.18225702, -0.18501348, -0.18333967,
       -0.18605884, -0.18626624, -0.18687093, -0.18803916, -0.19044883,
       -0.19076442, -0.19142873, -0.19278578, -0.19291074, -0.19453316,
       -0.19453203, -0.19408676, -0.19550697, -0.19721576, -0.19685972,
       -0.19727427, -0.19750051, -0.19905745, -0.19932915, -0.20025058])

In [6]:
# Kaplan-Yorke dimension

Lambda_sorted = sorted(Lambda, reverse=True)

sum_Lambda = 0

D_KY = 0

for Lambda_id in range(0, len(Lambda_sorted)):
    sum_Lambda = sum_Lambda + Lambda_sorted[Lambda_id]
    if sum_Lambda < 0:
        D_KY = Lambda_id - 1 - ((sum_Lambda - Lambda_sorted[Lambda_id])/Lambda_sorted[Lambda_id])
        break
        
D_KY = D_KY + 1

In [7]:
D_KY

2.165939376422042

In [8]:
np.savetxt('Lambda_MGactual', Lambda_sorted)